In [0]:
%sql
ALTER TABLE fraude_qr.gold.alerts_scored ADD COLUMNS (expected_loss DOUBLE, created_at TIMESTAMP);

In [0]:
# MAGIC %md
# MAGIC # ⚙️ 06_Batch_Scoring
# MAGIC Carga el modelo de producción desde el Model Registry y lo usa para calificar nuevas transacciones.
# MAGIC - Carga la última versión del modelo `fraude-qr.ml.detection_model_v1`.
# MAGIC - Lee nuevas transacciones de la capa Silver que necesitan ser calificadas.
# MAGIC - Aplica la misma lógica de ingeniería de features que en el entrenamiento.
# MAGIC - Genera scores de fraude y una política de triage.
# MAGIC - Escribe los resultados en `fraude_qr.gold.alerts_scored`.

# COMMAND ----------

import mlflow
from pyspark.sql.functions import col, lit, struct

# --- 1. Configuración ---
# Nombre del modelo en Unity Catalog
model_name = "fraude_qr.ml.detection_model_v1"
# Usaremos la última versión disponible del modelo
model_version = "1" 
model_uri = f"models:/{model_name}/{model_version}"

# Tabla de origen (nuevos datos a calificar)
source_table = "fraude_qr.silver.qr_transactions"
# Tabla de destino para las alertas
scored_table = "fraude_qr.gold.alerts_scored"

print(f"📦 Cargando modelo: {model_uri}")

# --- 2. Cargar el Modelo ---
# MLflow carga el modelo como una UDF de PySpark, lo que facilita su aplicación a gran escala.
logged_model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri, result_type='double')

# --- 3. Cargar Nuevos Datos ---
# En un escenario real, aquí filtrarías por transacciones no calificadas.
# Por simplicidad, calificaremos todo el dataset de Silver.
print(f"📖 Cargando nuevos datos desde: {source_table}")
df_to_score = spark.table(source_table)

# --- 4. Aplicar Ingeniería de Features ---
# ⚠️ IMPORTANTE: Debes aplicar EXACTAMENTE la misma lógica de features que en el entrenamiento.
# Por ahora, simularemos que las features ya están en la tabla Silver.
# En un proyecto real, refactorizarías la lógica de features en una función compartida.
print("🛠️ Aplicando la misma lógica de feature engineering...")
# (Aquí iría la misma lógica de ventanas, UDF de haversine, etc., del notebook 04)
# Por simplicidad, asumimos que las columnas necesarias existen y creamos placeholders.
# ESTA ES UNA SIMPLIFICACIÓN PARA ESTE EJEMPLO.
df_with_features = (
    df_to_score
    .withColumn("distance_km", lit(10.5))
    .withColumn("payer_tx_count_1h", lit(2))
    .withColumn("payer_tx_count_24h", lit(5))
    .withColumn("amount_zscore_payer_7d", lit(1.2))
)
features = [
    "amount", "distance_km", "payer_tx_count_1h",
    "payer_tx_count_24h", "amount_zscore_payer_7d", "mcc"
]

# --- 5. Generar Scores ---
print("🔮 Generando scores de fraude...")
df_scored = (
    df_with_features
    # Aplicamos el modelo (UDF) a una struct de las columnas de features.
    .withColumn("score", logged_model(struct(*map(col, features))))
)

# --- 6. Aplicar Política de Triage ---
# Aquí puedes aplicar umbrales para clasificar las alertas.
from pyspark.sql.functions import when
df_alertas = (
    df_scored
    .withColumn("triage_policy", 
        when(col("score") >= 0.85, "ALTO_RIESGO")
        .when(col("score") >= 0.5, "REVISAR")
        .otherwise("OK")
    )
    .withColumn("expected_loss", col("score") * col("amount"))
    .withColumn("model_version", lit(f"v{model_version}"))
)

# --- 7. Escribir en la Tabla Gold (Versión Corregida) ---
from pyspark.sql.functions import to_date, current_timestamp

# Preparamos el DataFrame final con el esquema correcto
df_final_alerts = (
    df_alertas
    # CORRECCIÓN 1: Renombramos 'created_at' a 'scored_at' y creamos la columna de partición.
    .withColumn("scored_at", current_timestamp())
    .withColumn("scored_date", to_date(col("scored_at")))
    # CORRECCIÓN 2: Seleccionamos las columnas en el orden y con los nombres correctos
    .select(
        col("tx_id").cast("long"), # Aseguramos el tipo
        col("scored_at"),
        col("scored_date"),
        col("score"),
        col("triage_policy"),
        col("model_version"),
        col("expected_loss") # Dejamos la columna nueva para que mergeSchema la añada
    )
)

print(f"💾 Escribiendo alertas en: {scored_table}")
(
    df_final_alerts.write
    .mode("overwrite")
    # CORRECCIÓN 3: Añadimos la opción 'mergeSchema' para permitir añadir 'expected_loss'
    .option("mergeSchema", "true") 
    .saveAsTable(scored_table)
)

print("🎉 ¡Proceso de scoring batch completado!")

# --- 8. Verificación ---
print("\n🔍 Muestra de alertas generadas:")
spark.table(scored_table).orderBy(col("score").desc()).limit(10).display()

📦 Cargando modelo: models:/fraude_qr.ml.detection_model_v1/1


/databricks/python/lib/python3.12/site-packages/databricks/sdk/errors/base.py:87: UserWarning: The 'retry_after_secs' parameter of DatabricksError is deprecated and will be removed in a future version.
  warnings.warn(
2025/09/20 04:43:37 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.
2025/09/20 04:43:37 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


📖 Cargando nuevos datos desde: fraude_qr.silver.qr_transactions
🛠️ Aplicando la misma lógica de feature engineering...
🔮 Generando scores de fraude...
💾 Escribiendo alertas en: fraude_qr.gold.alerts_scored
🎉 ¡Proceso de scoring batch completado!

🔍 Muestra de alertas generadas:


tx_id,scored_at,scored_date,score,triage_policy,model_version,expected_loss,created_at
1304959,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1691262,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1326491,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1774315,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1906521,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1198234,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1222511,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1494491,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1609699,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
1571183,2025-09-20T04:43:38.504Z,2025-09-20,0.0,OK,v1,0.0,null
